## Imports

In [13]:
from __future__ import print_function
import keras
import os
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

## Parameters

In [2]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

## Load Data

In [3]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Train Model

In [4]:
model_path = "./model_weights.h5"
if os.path.exists(model_path):
    model = load_model(model_path)
    print("Loaded model...")
else:
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    
    model.save(model_path)
    print("Saved model...")

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Loaded model...
Test loss: 0.03083601578749367
Test accuracy: 0.9899


## Train Model (Split)

In [14]:
x_train_seperated = {}
y_train_seperated = {}
for target_label in range(10):
    x_train_seperated[target_label] = []
    y_train_seperated[target_label] = []
    for i, label in enumerate(y_train):
        if list(label).index(1) == target_label:
            x_train_seperated[target_label].append(x_train[i])
            y_train_seperated[target_label].append(y_train[i])
            
x_test_seperated = {}
y_test_seperated = {}
for target_label in range(10):
    x_test_seperated[target_label] = []
    y_test_seperated[target_label] = []
    for i, label in enumerate(y_test):
        if list(label).index(1) == target_label:
            x_test_seperated[target_label].append(x_test[i])
            y_test_seperated[target_label].append(y_test[i])
            
x_train_seperated = [np.array(x_train_seperated[i]) for i in range(10)]
y_train_seperated = [np.array(y_train_seperated[i]) for i in range(10)]
x_test_seperated = [np.array(x_test_seperated[i]) for i in range(10)]
y_test_seperated = [np.array(y_test_seperated[i]) for i in range(10)]

In [ ]:
# EXAMPLE from https://stackoverflow.com/questions/46772685/how-to-accumulate-gradients-in-tensorflow

## Optimizer definition - nothing different from any classical example
opt = tf.train.AdamOptimizer()

## Retrieve all trainable variables you defined in your graph
tvs = tf.trainable_variables()
## Creation of a list of variables with the same shape as the trainable ones
# initialized with 0s
accum_vars = [tf.Variable(tf.zeros_like(tv.initialized_value()), trainable=False) for tv in tvs]
zero_ops = [tv.assign(tf.zeros_like(tv)) for tv in accum_vars]

## Calls the compute_gradients function of the optimizer to obtain... the list of gradients
gvs = opt.compute_gradients(rmse, tvs)

## Adds to each element from the list you initialized earlier with zeros its gradient (works because accum_vars and gvs are in the same order)
accum_ops = [accum_vars[i].assign_add(gv[0]) for i, gv in enumerate(gvs)]

## Define the training step (part with variable value update)
train_step = opt.apply_gradients([(accum_vars[i], gv[1]) for i, gv in enumerate(gvs)])



## The while loop for training
while ...:
    # Run the zero_ops to initialize it
    sess.run(zero_ops)
    # Accumulate the gradients 'n_minibatches' times in accum_vars using accum_ops
    for i in xrange(n_minibatches):
        sess.run(accum_ops, feed_dict=dict(X: Xs[i], y: ys[i]))
    # Run the train_step ops to update the weights based on your accumulated gradients
    sess.run(train_step)

In [16]:
model_split = load_model(model_path)

# train model on every class, iterately (one client per class)
for label in range(10):
    print('Training on client {} using class {}'.format(label, label))
    model_split.fit(x_train_seperated[label], y_train_seperated[label],
              batch_size=batch_size,
              epochs=1,
              verbose=1,
              validation_data=(x_test, y_test))

Training on client 0 using class 0
Train on 5923 samples, validate on 10000 samples
Epoch 1/1
5923/5923 [==============================] - 10s 2ms/step - loss: 0.0030 - acc: 0.9993 - val_loss: 0.0787 - val_acc: 0.9785
Training on client 1 using class 1
Train on 6742 samples, validate on 10000 samples
Epoch 1/1
6742/6742 [==============================] - 13s 2ms/step - loss: 0.0044 - acc: 0.9987 - val_loss: 0.1955 - val_acc: 0.9495
Training on client 2 using class 2
Train on 5958 samples, validate on 10000 samples
Epoch 1/1
5958/5958 [==============================] - 12s 2ms/step - loss: 0.0212 - acc: 0.9953 - val_loss: 0.1860 - val_acc: 0.9480
Training on client 3 using class 3
Train on 6131 samples, validate on 10000 samples
Epoch 1/1
6131/6131 [==============================] - 13s 2ms/step - loss: 0.0247 - acc: 0.9936 - val_loss: 0.3797 - val_acc: 0.8882
Training on client 4 using class 4
Train on 5842 samples, validate on 10000 samples
Epoch 1/1
5842/5842 [=======================

## Train Model (Label Poisoning)

In [6]:
model_path = "./model_weights.h5"
adv_model = load_model(model_path)
print("Loaded model...")

adv_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

adv_model_path = "./adv_model_weights.h5"
adv_model.save(adv_model_path)
print("Saved model...")

score = adv_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Loaded model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 5s 82us/step - loss: 0.0239 - accuracy: 0.9924 - val_loss: 0.0235 - val_accuracy: 0.9932
Epoch 2/12
23168/60000 [==========>...................] - ETA: 2s - loss: 0.0215 - accuracy: 0.9921

KeyboardInterrupt: 